In [1]:
from parser import get_files_in_folder, parse_gene_sequences
from damerau_levenshtein import damerau_levenshtein
from gen_algo import crossover, mutate

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# import umap

import os, sys

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.cluster import KMeans, HDBSCAN, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.model_selection import ParameterGrid
from keras.utils import to_categorical

ImportError: cannot import name 'KerasLazyLoader' from 'tensorflow.python.util.lazy_loader' (c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\lazy_loader.py)

In [ ]:
dataset = pd.read_csv('dataset copy.csv', sep=';')

x = dataset.iloc[:, 1:].values
y = dataset.iloc[:, 0].values

In [ ]:
pca = PCA(n_components=0.95)
x_pca = pca.fit_transform(x)

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_pca)

In [ ]:
# implementing silhoutte score
params = [2, 3, 4, 5, 6, 7, 8, 9, 10]
paramGrid = ParameterGrid({'n_clusters': params})

best_score = -1

silhouette_scores = []

kmeans = KMeans()

for p in paramGrid:
	kmeans.set_params(**p)
	kmeans.fit(x)

	silScore = silhouette_score(x, kmeans.labels_, random_state=0)
	silhouette_scores.append(silScore)

	print('Parameter: ', p, 'Score: ', silScore)

	if silScore > best_score:
		best_score = silScore
		best_grid = p

In [ ]:
# plot the silhouette scores
plt.bar(range(len(silhouette_scores)), silhouette_scores, tick_label=list(params))
plt.xticks(range(len(silhouette_scores)), params)
plt.title('Silhouette Scores for Different Parameters')
plt.xlabel('nb of clusters')

plt.show()

In [ ]:
print('Best grid: ', best_grid['n_clusters'], 'Best score: ', best_score)

y_pred = KMeans(n_clusters=best_grid['n_clusters'], random_state=0).fit_predict(x)

km = KMeans(n_clusters=best_grid['n_clusters'], random_state=0).fit(x)
y_kmeans = km.predict(x)

plt.scatter(x[:, 0], x[:, 1], c=kmeans.labels_)

# legend
for i in range(best_grid['n_clusters']):
	plt.scatter([], [], color=plt.cm.tab10(i), label=f'Cluster {i}')
plt.legend()

plt.title('PCA')
plt.xlabel('PCA1')
plt.ylabel('PCA2')
plt.show()

# 3D plot interactive
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.view_init(azim=0)
ax.scatter(x[:, 0], x[:, 1], x[:, 2], c=kmeans.labels_)
# legend
for i in range(best_grid['n_clusters']):
	ax.scatter([], [], [], color=plt.cm.tab10(i), label=f'Cluster {i}')
ax.legend()
ax.set_title('3D PCA')
ax.set_xlabel('PCA1')
ax.set_ylabel('PCA2')
ax.set_zlabel('PCA3')
plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

pca_3d = PCA(n_components=3)
X_encoded_3d = pca_3d.fit_transform(X_encoded)

fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111, projection='3d')

# Use y_pred from kmeans clustering on X_encoded
for i in np.unique(y_pred):
	ax.scatter(
		X_encoded_3d[y_pred == i, 0],
		X_encoded_3d[y_pred == i, 1],
		X_encoded_3d[y_pred == i, 2],
		s=40,
		label=f'Cluster {i}'
	)

ax.set_title('3D Representation of Clusters')
ax.set_xlabel('PC1')
ax.set_ylabel('PC2')
ax.set_zlabel('PC3')
ax.legend()
plt.show()

In [ ]:
# ax = plt.gca()
# ax.set_xlim([-200, 450])
# ax.set_ylim([-100, 80])

plt.scatter(x[y_kmeans == 0, 0], x[y_kmeans == 0, 1], s=50, c='red', label='Cluster 1')
plt.scatter(x[y_kmeans == 1, 0], x[y_kmeans == 1, 1], s=50, c='blue', label='Cluster 2')
# name the axes
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Clusters of proteins')
plt.legend()

plt.show()

# print the number of points in each cluster
unique, counts = np.unique(y_kmeans, return_counts=True)
print("Number of points in each cluster:")
for i, count in zip(unique, counts):
    print(f"Cluster {i}: {count} points")
    

In [ ]:
hdb = HDBSCAN()
hdb.fit(x)
hdb_labels = hdb.labels_

In [ ]:
# print max number of clusters found by HDBSCAN
print('Max number of clusters found by HDBSCAN: ', len(np.unique(hdb_labels)))

# ax = plt.gca()
# ax.set_ylim([-10, 20])
# ax.set_xlim([-10, 20])

for i in np.unique(hdb_labels):
	plt.scatter(x[hdb_labels == i, 0], x[hdb_labels == i, 1], s=50, label='Cluster ' + str(i))

# plt.legend()

plt.title('Clusters of proteins')
plt.xlabel
plt.ylabel('Y')

# plt.legend()
plt.show()

In [ ]:
# Définir un modèle autoencodeur pour le clustering
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(x.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(x.shape[1], activation='sigmoid'))

# Compiler le modèle avec une fonction de perte adaptée
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Afficher le résumé du modèle
model.summary()
                                                                       
# Entraînement du modèle (Autoencodeur)
history = model.fit(x, x, epochs=100, batch_size=16, verbose=0)

# Afficher l'évolution de la fonction de perte
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.show()

# Afficher l'évolution de la précision
plt.plot(history.history['accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.show()

In [ ]:
# clustering
encoder = Sequential()
encoder.add(model.layers[0])
encoder.add(model.layers[1])
encoder.add(model.layers[2])

X_encoded = encoder.predict(x)

In [ ]:
# clustering
kmeans = KMeans(n_clusters=2, random_state=0)
y_pred = kmeans.fit_predict(X_encoded)

# ax = plt.gca()
# ax.set_xlim([-5, 50])
# ax.set_ylim([-5, 30])

# Afficher les résultats du clustering
for i in range(2):
	plt.scatter(X_encoded[y_pred == i, 0], X_encoded[y_pred == i, 1], s=50, label='Cluster ' + str(i))

plt.title('Clusters of Proteins')
plt.xlabel('X1')
plt.ylabel('X2')
plt.legend()

plt.show()

In [ ]:
dbscan_2 = DBSCAN(min_samples=5)
dbscan_labels_2 = dbscan_2.fit_predict(X_encoded)

# print max number of clusters found by DBSCAN
print('Max number of clusters found by HDBSCAN: ', len(np.unique(dbscan_labels_2)))

# ax = plt.gca()
# ax.set_xlim([-20, 50])
# ax.set_ylim([-30, 20])

for i in range(len(np.unique(dbscan_labels_2))):
	plt.scatter(X_encoded[dbscan_labels_2 == i, 0], X_encoded[dbscan_labels_2 == i, 1], s=50, label='Cluster ' + str(i))

plt.title('Clusters of Proteins')
plt.xlabel('X1')
plt.ylabel('X2')
# plt.legend()
plt.show()